<a href="https://colab.research.google.com/github/alslemos/MyMaps/blob/main/Manipulando_Sidra.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install sidrapy

In [ ]:
#importações

import sidrapy
import pandas as pd

In [15]:
#chamada de funcao pelo ibge

query = sidrapy.get_table(
    table_code='4709', #table code
    territorial_level='6', # 6 eh municipio
    ibge_territorial_code='all',  #pode filtrar se quiser
    period='all',                 # periodo
    variable='all',           # pode ser absoluto, variando etc
  )

print(query.shape)
query.head()


(16711, 11)


,NC,NN,MC,MN,V,D1C,D1N,D2C,D2N,D3C,D3N
0,Nível Territorial (Código),Nível Territorial,Unidade de Medida (Código),Unidade de Medida,Valor,Município (Código),Município,Ano (Código),Ano,Variável (Código),Variável
1,6,Município,45,Pessoas,21494,1100015,Alta Floresta D'Oeste - RO,2022,2022,93,População residente
2,6,Município,45,Pessoas,-2898,1100015,Alta Floresta D'Oeste - RO,2022,2022,5936,Variação absoluta da população residente 2010 ...
3,6,Município,2,%,-1.05,1100015,Alta Floresta D'Oeste - RO,2022,2022,10605,Taxa de crescimento geométrico
4,6,Município,45,Pessoas,96833,1100023,Ariquemes - RO,2022,2022,93,População residente


In [16]:
# Tratamentos de acentos
query = query.apply(lambda x: x.str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8') if x.dtype == 'object' else x)

# removendo primeira linha
query = query.iloc[1:]

# mostrando como está
query.head()

,NC,NN,MC,MN,V,D1C,D1N,D2C,D2N,D3C,D3N
1,6,Municipio,45,Pessoas,21494,1100015,Alta Floresta D'Oeste - RO,2022,2022,93,Populacao residente
2,6,Municipio,45,Pessoas,-2898,1100015,Alta Floresta D'Oeste - RO,2022,2022,5936,Variacao absoluta da populacao residente 2010 ...
3,6,Municipio,2,%,-1.05,1100015,Alta Floresta D'Oeste - RO,2022,2022,10605,Taxa de crescimento geometrico
4,6,Municipio,45,Pessoas,96833,1100023,Ariquemes - RO,2022,2022,93,Populacao residente
5,6,Municipio,45,Pessoas,6480,1100023,Ariquemes - RO,2022,2022,5936,Variacao absoluta da populacao residente 2010 ...


In [18]:
#pivotando tabela
query = query.pivot_table(index=['D1C','D1N'], columns='D3N', values = 'V').reset_index()
query.columns.name = None

print(query.shape)
query.head()

(5570, 5)


,D1C,D1N,Populacao residente,Taxa de crescimento geometrico,Variacao absoluta da populacao residente 2010 compatibilizada
0,1100015,Alta Floresta D'Oeste - RO,21494.0,-1.05,-2898.0
1,1100023,Ariquemes - RO,96833.0,0.58,6480.0
2,1100031,Cabixi - RO,5351.0,-1.37,-962.0
3,1100049,Cacoal - RO,86887.0,0.84,8313.0
4,1100056,Cerejeiras - RO,15890.0,-0.58,-1139.0


In [19]:
column_mapping =  {
     'D1C': 'CD_MUN',
     'D1N': 'NM_MUN',
     'Populacao residente' : 'POP_RESIDENTE',
     'Variacao absoluta da populacao residente 2010 compatibilizada' : 'VAR_ABS',
     'Taxa de crescimento geometrico' : 'TX_CRESC'
}

#renomeando colunas
query.rename(columns=column_mapping, inplace=True)

print(query.shape)
query.head()

(5570, 5)


,CD_MUN,NM_MUN,POP_RESIDENTE,TX_CRESC,VAR_ABS
0,1100015,Alta Floresta D'Oeste - RO,21494.0,-1.05,-2898.0
1,1100023,Ariquemes - RO,96833.0,0.58,6480.0
2,1100031,Cabixi - RO,5351.0,-1.37,-962.0
3,1100049,Cacoal - RO,86887.0,0.84,8313.0
4,1100056,Cerejeiras - RO,15890.0,-0.58,-1139.0


In [20]:
query.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5570 entries, 0 to 5569
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   CD_MUN         5570 non-null   object
 1   NM_MUN         5570 non-null   object
 2   POP_RESIDENTE  5570 non-null   object
 3   TX_CRESC       5570 non-null   object
 4   VAR_ABS        5570 non-null   object
dtypes: object(5)
memory usage: 217.7+ KB


In [21]:
#trocando tipos de valores das colunas
query[['CD_MUN', 'POP_RESIDENTE', 'VAR_ABS']] = query[['CD_MUN', 'POP_RESIDENTE', 'VAR_ABS']].astype(int)
query['TX_CRESC'] = query['TX_CRESC'].astype(float)

query.head()

,CD_MUN,NM_MUN,POP_RESIDENTE,TX_CRESC,VAR_ABS
0,1100015,Alta Floresta D'Oeste - RO,21494,-1.05,-2898
1,1100023,Ariquemes - RO,96833,0.58,6480
2,1100031,Cabixi - RO,5351,-1.37,-962
3,1100049,Cacoal - RO,86887,0.84,8313
4,1100056,Cerejeiras - RO,15890,-0.58,-1139
